## 環境変数の設定

In [1]:
import os

# 作業ディレクトリをcloud functionsに合わせる
os.chdir("../function")

# 環境変数を設定
os.environ["ENV_FOR_DYNACONF"] = "staging"
# os.environ["ENV_FOR_DYNACONF"] = "production"

# 環境変数を確認
print(os.environ["ENV_FOR_DYNACONF"])

staging


## configファイルの読み込み

In [2]:
import json

from dynaconf import Dynaconf


SETTINGS = Dynaconf(
    environments=True,
    settings_files="settings.toml",
)

print(json.dumps(SETTINGS.as_dict(), indent=2, ensure_ascii=False))

{
  "PROJECT_NAME": "nursery-vacanc-notifier",
  "REQUEST_URL": {
    "習志野市": "https://www.city.narashino.lg.jp/soshiki/kodomo_hoiku/gyomu/hoikugakko/akireigetu.html"
  },
  "LINE_NOTIFY_API": "https://notify-api.line.me/api/notify",
  "SECRET_MANAGER_NAME": "LINE_NOTIFY",
  "LINE_NOTIFY_MAX_MESSAGE_LENGTH": 1000,
  "TARGET_AGE": [
    "0歳児",
    "1歳児",
    "2歳児"
  ],
  "TARGET_AVAILABILITY": [
    "1～2人",
    "3人以上"
  ],
  "AVAILABILITY": {
    "◯": "3人以上",
    "△": "1～2人",
    "×": "空きなし",
    "―": "受け入れなし"
  },
  "LINE_NOTIFY_TOKEN_ID": "line_notify_dev"
}


In [3]:
from utils.dynaconf import get_config_value


get_config_value("LINE_NOTIFY_TOKEN_ID")

'line_notify_dev'

## 保育所等空き状況の取得

In [4]:
from datetime import datetime

import pandas as pd
import requests
from bs4 import BeautifulSoup


pd.set_option("future.no_silent_downcasting", True)


def read_html_tables(html_source: bytes) -> pd.DataFrame:
    """Read HTML tables into pandas DataFrames and clean the first table.

    Args:
        html_source (bytes): html content to be parsed

    Returns:
        pd.DataFrame: cleaned DataFrame
    """
    df = pd.read_html(html_source)[0]
    df.drop([0, 1], inplace=True)
    df.columns = ["名称1", "名称", "定員", "0歳児", "1歳児", "2歳児", "3歳児", "4歳児", "5歳児"]

    # 名称を結合して、不要な列を削除
    df["名称"] = df["名称"] + "(定員:" + df["定員"] + ")"
    df.drop(["名称1", "定員"], axis=1, inplace=True)

    # 空き状況の記号を文字列に変換
    df = df.replace(SETTINGS["availability"])

    # フォーマット変更
    df = df.melt(id_vars=["名称"], var_name="年齢", value_name="空き")
    df = df[df["空き"].isin(SETTINGS["target_availability"])]
    return df[df["年齢"].isin(SETTINGS["target_age"])]

def extract_update_time(html_source: bytes) -> str:
    """Extract the update time from the HTML source.

    Args:
        html_source (bytes): HTML source to be parsed

    Returns:
        str: update time in the format of %a, %d %b %Y %H:%M:%S GMT
    """
    soup = BeautifulSoup(html_source, 'html.parser')
    meta_tag = soup.find('meta', attrs={'name': 'nsls:timestamp'})
    timestamp_str = meta_tag['content'] if meta_tag else 'Tag not found'
    return datetime.strptime(timestamp_str, '%a, %d %b %Y %H:%M:%S GMT')


for city in SETTINGS["request_url"]:
    html_source = requests.get(SETTINGS["request_url"][city], timeout=10)
    update_time = extract_update_time(html_source.text)
    print(update_time)
    df = read_html_tables(html_source.content)
    print(df)

2024-05-25 00:00:00
                                名称   年齢    空き
0                    藤崎保育所(定員:123)  0歳児  1～2人
2                 本大久保第二保育所(定員:47)  0歳児  1～2人
10                  向山こども園(定員:142)  0歳児  3人以上
13               明徳そでにの保育園(定員:110)  0歳児  3人以上
17                谷津みのり保育園(定員:138)  0歳児  1～2人
32                   青葉幼稚園(定員:121)  0歳児  1～2人
35               ひまわり保育園2nd(定員:18)  0歳児  3人以上
36               ひまわり保育園3rd(定員:18)  0歳児  1～2人
37          サンライズキッズ保育園津田沼園(定員:19)  0歳児  1～2人
38          サンライズキッズ保育園奏の杜園(定員:18)  0歳児  1～2人
39                  ひまわり保育園(定員:18)  0歳児  1～2人
42                  ロゼッタ保育園(定員:18)  0歳児  3人以上
43   ポピンズナーサリースクールイオンモール津田沼(定員:18)  0歳児  3人以上
44              みらいつむぎ谷津保育園(定員:19)  0歳児  1～2人
45              ひまわり保育園Sola(定員:19)  0歳児  3人以上
47                   谷津保育所(定員:109)  1歳児  3人以上
50                  谷津南保育所(定員:160)  1歳児  3人以上
53                 袖ケ浦こども園(定員:125)  1歳児  3人以上
56                  向山こども園(定員:142)  1歳児  3人以上
57                   かすみ保育園(定員:90)  1歳児  1～2人
58            

In [6]:
from scraping import scrape


scrape()

(                                名称   年齢    空き
 0                    藤崎保育所(定員:123)  0歳児  1～2人
 2                 本大久保第二保育所(定員:47)  0歳児  1～2人
 10                  向山こども園(定員:142)  0歳児  3人以上
 13               明徳そでにの保育園(定員:110)  0歳児  3人以上
 17                谷津みのり保育園(定員:138)  0歳児  1～2人
 32                   青葉幼稚園(定員:121)  0歳児  1～2人
 35               ひまわり保育園2nd(定員:18)  0歳児  3人以上
 36               ひまわり保育園3rd(定員:18)  0歳児  1～2人
 37          サンライズキッズ保育園津田沼園(定員:19)  0歳児  1～2人
 38          サンライズキッズ保育園奏の杜園(定員:18)  0歳児  1～2人
 39                  ひまわり保育園(定員:18)  0歳児  1～2人
 42                  ロゼッタ保育園(定員:18)  0歳児  3人以上
 43   ポピンズナーサリースクールイオンモール津田沼(定員:18)  0歳児  3人以上
 44              みらいつむぎ谷津保育園(定員:19)  0歳児  1～2人
 45              ひまわり保育園Sola(定員:19)  0歳児  3人以上
 47                   谷津保育所(定員:109)  1歳児  3人以上
 50                  谷津南保育所(定員:160)  1歳児  3人以上
 53                 袖ケ浦こども園(定員:125)  1歳児  3人以上
 56                  向山こども園(定員:142)  1歳児  3人以上
 57                   かすみ保育園(定員:90)  1歳児  1～2人
 58          

## GCPの認証

In [ ]:
!gcloud config set project
!gcloud auth application-default login

## GCPのシークレットマネージャーからAPIキーを取得

In [ ]:
from google.cloud import secretmanager


# クライアントの初期化
client = secretmanager.SecretManagerServiceClient()

# # 秘密情報の名前を指定
name = f"projects/{SETTINGS['gcp_project_id']}/secrets/{SETTINGS['secret_manager_name']}/versions/latest"

# # 秘密情報の取得
response = client.access_secret_version(name=name)
secret = json.loads(response.payload.data.decode("UTF-8"))
secret

In [ ]:
from function.gcp import gcp_secretmanager


gcp_secretmanager()

## line に通知する

In [5]:
from datetime import datetime

import pytz
import requests


# 現在の日時を日本時間で取得
jst = pytz.timezone("Asia/Tokyo")
today = datetime.now(tz=jst).strftime("%Y年%m月%d日")


def line_notify(message: str) -> None:
    headers = {"Authorization": f"Bearer {secret[SETTINGS['line_notify_token_id']]}"}
    data = {"message": f"\n{message}"}
    requests.post(SETTINGS["line_notify_api"], headers=headers, data=data, timeout=10)


for city in SETTINGS["request_url"]:
    notify_message = f"🏢{city}🏢({today}時点)\n"
    notify_message += f"{SETTINGS['request_url'][city]}"
    line_notify(notify_message)

    for age in SETTINGS["target_age"]:
        for availability in SETTINGS["target_availability"]:
            notify_message = f"👶{age}で{availability}空きあり👶\n"
            df_target = df[(df["空き"] == availability) & (df["年齢"] == age)]

            for _, row in df_target.iterrows():
                # 次に追加する文字列
                next_part = f"🏡{row['名称']}\n"
                # 現在のメッセージに追加すると100文字を超える場合は、現在のメッセージを送信
                if len(notify_message + next_part) > SETTINGS["line_notify_max_message_length"]:
                    line_notify(notify_message)
                    # print(notify_message)
                    notify_message = f"【{age}の空き状況】\n"  # 新しいヘッダーを追加
                # 文字列を追加
                notify_message += next_part

            # 最後のメッセージが残っていれば送信
            if notify_message:
                line_notify(notify_message)
                # print(notify_message)

In [1]:
from loguru import logger


logger.trace("トレース")
logger.debug("デバッグ")
logger.info("情報")
logger.success("成功")
logger.warning("警告")
logger.error("エラー")
logger.critical("クリティカル")

2024-05-06 22:14:10.809 | DEBUG    | __main__:<module>:5 - デバッグ
2024-05-06 22:14:10.811 | INFO     | __main__:<module>:6 - 情報
2024-05-06 22:14:10.813 | SUCCESS  | __main__:<module>:7 - 成功
2024-05-06 22:14:10.816 | WARNING  | __main__:<module>:8 - 警告
2024-05-06 22:14:10.818 | ERROR    | __main__:<module>:9 - エラー
2024-05-06 22:14:10.820 | CRITICAL | __main__:<module>:10 - クリティカル
